In [ ]:
import requests
import pandas as pd
from datetime import datetime
from datetime import timedelta
import time

daily_crypto_starting_day = '01-01-2020'
daily_crypto_starting_date = datetime.strptime(daily_crypto_starting_day, '%m-%d-%Y')

daily_crypto_ending_day = '01-17-2022'
daily_crypto_ending_date = datetime.strptime(daily_crypto_ending_day, '%m-%d-%Y')

current_date = daily_crypto_starting_date
rates_df = None
while current_date <= daily_crypto_ending_date :
  current_date_string = current_date.strftime('%m-%d-%Y')
  print(current_date_string)
  try:
    url = 'https://aave-api-v2.aave.com/data/liquidity/v2?poolId=0xb53c1a33016b2dc2ff3653530bff1848a515c8c5&date=' + current_date_string
    r = requests.get(url)
    j = r.json()

    df = pd.DataFrame.from_dict(j)

    daily_df = df[['name', 'variableBorrowRate', 'stableBorrowRate', 'liquidityRate', 'avg1DaysVariableBorrowRate',
                   'avg7DaysVariableBorrowRate', 'avg30DaysVariableBorrowRate', 'avg91DaysVariableBorrowRate',
                   'avg1DaysLiquidityRate', 'avg7DaysLiquidityRate', 'avg30DaysLiquidityRate',
                   'avg91DaysLiquidityRate']]
    daily_df = daily_df.sort_values(by=['name'])
    daily_df['Date'] = current_date


    if rates_df is None :
        rates_df = daily_df.copy()
    else:
        rates_df = pd.concat([rates_df, daily_df])

    time.sleep(2)
  except : 
    print(f'trouble fetching {current_date_string}')
  current_date = current_date + timedelta(days = 1)

print('all data fetched')
print(rates_df.shape)


01-01-2020
01-02-2020
01-03-2020
01-04-2020
01-05-2020
01-06-2020
01-07-2020
01-08-2020
01-09-2020
01-10-2020
01-11-2020
01-12-2020
01-13-2020
01-14-2020
01-15-2020
01-16-2020
01-17-2020
01-18-2020
01-19-2020
01-20-2020
01-21-2020
01-22-2020
01-23-2020
01-24-2020
01-25-2020
01-26-2020
01-27-2020
01-28-2020
01-29-2020
01-30-2020
01-31-2020
02-01-2020
02-02-2020
02-03-2020
02-04-2020
02-05-2020
02-06-2020
02-07-2020
02-08-2020
02-09-2020
02-10-2020
02-11-2020
02-12-2020
02-13-2020
02-14-2020
02-15-2020
02-16-2020
02-17-2020
02-18-2020
02-19-2020
02-20-2020
02-21-2020
02-22-2020
02-23-2020
02-24-2020
02-25-2020
02-26-2020
02-27-2020
02-28-2020
02-29-2020
03-01-2020
03-02-2020
03-03-2020
03-04-2020
03-05-2020
03-06-2020
03-07-2020
03-08-2020
03-09-2020
03-10-2020
03-11-2020
03-12-2020
03-13-2020
03-14-2020
03-15-2020
03-16-2020
03-17-2020
03-18-2020
03-19-2020
03-20-2020
03-21-2020
03-22-2020
03-23-2020
03-24-2020
03-25-2020
03-26-2020
03-27-2020
03-28-2020
03-29-2020
03-30-2020
03-31-2020

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
rates_df.to_pickle('/content/gdrive/MyDrive/AaveData/rates_df.pkl')

In [ ]:
rates_df = pd.read_pickle('/content/gdrive/MyDrive/AaveData/rates_df.pkl')

In [ ]:
rates_df

,name,variableBorrowRate,stableBorrowRate,liquidityRate,avg1DaysVariableBorrowRate,avg7DaysVariableBorrowRate,avg30DaysVariableBorrowRate,avg91DaysVariableBorrowRate,avg1DaysLiquidityRate,avg7DaysLiquidityRate,avg30DaysLiquidityRate,avg91DaysLiquidityRate,Date
29,0x Protocol Token,0.00246641078358969225,0.03408720076640415241,0.00040890023444781115,0,0,0,0,0,0,0,0,2020-01-01
14,Aave Token,0,0,0,0,0,0,0,0,0,0,0,2020-01-01
24,Ampleforth,0.01173444541554875681,0,0.00069998763164798702,0,0,0,0,0,0,0,0,2020-01-01
21,Balancer,0.06413700305234542721,0.12615133491726173341,0.02007276093596930928,0,0,0,0,0,0,0,0,2020-01-01
4,Basic Attention Token,0.01367299925963052625,0.05064107404201471532,0.00103998753524218743,0,0,0,0,0,0,0,0,2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,Uniswap,0.00291646488205642672,0.00500486025095391799,0.00004361638567585159,0.0013309019567712,0.00165510275175792,0.00697860423535104,0.01198294768929888,0.0000124112340504,0.0000973967474016,0.00112060533503952,0.0012715948222128,2022-01-17
8,Wrapped BTC,0.0027003402777472723,0.03393392954477999304,0.00004786486305112304,0.00259605294201072,0.00257260511774304,0.00284381754494544,0.00422206858526688,0.00004441405609728,0.00006093974728656,0.000085208174856,0.00027381925666512,2022-01-17
23,Wrapped Ether,0.00377203062534592459,0.03531545521627415242,0.00010900715416202143,0.00354861258748128,0.00335756902267008,0.00299605842222624,0.00296651193354288,0.00011280910299984,0.00010880965134576,0.00011550117111696,0.00102930890517072,2022-01-17
25,renFIL,0.03543473505110974439,0,0.00904803893874110808,0.11264125889374416,0.0354813909746256,0.02863690059540384,0.0187655795709624,0.02716599882336192,0.00818588816801136,0.0060978658011696,0.00317495063454048,2022-01-17


Processing the data

In [ ]:
values_of_interest = ['variableBorrowRate', 'stableBorrowRate', 'liquidityRate', 'avg1DaysVariableBorrowRate',
                   'avg7DaysVariableBorrowRate', 'avg30DaysVariableBorrowRate', 'avg91DaysVariableBorrowRate',
                   'avg1DaysLiquidityRate', 'avg7DaysLiquidityRate', 'avg30DaysLiquidityRate',
                   'avg91DaysLiquidityRate']
processed_df = None
for value_of_interest in values_of_interest:
  melted_df = pd.melt(rates_df, id_vars=['name','Date'], value_vars=value_of_interest)
  melted_df = melted_df.rename(columns = {'value':value_of_interest})
  melted_df = melted_df.drop(columns=['variable'])
  if processed_df is None:
    processed_df = melted_df.copy()
  else :
    processed_df = pd.merge(processed_df, melted_df, left_on=['name','Date'], right_on=['name','Date'])
print('plot')

plot


In [ ]:
processed_df.head()

,name,Date,variableBorrowRate,stableBorrowRate,liquidityRate,avg1DaysVariableBorrowRate,avg7DaysVariableBorrowRate,avg30DaysVariableBorrowRate,avg91DaysVariableBorrowRate,avg1DaysLiquidityRate,avg7DaysLiquidityRate,avg30DaysLiquidityRate,avg91DaysLiquidityRate
0,0x Protocol Token,2020-01-01,0.00246641078358969225,0.03408720076640415241,0.00040890023444781115,0,0,0,0,0,0,0,0
1,Aave Token,2020-01-01,0,0,0,0,0,0,0,0,0,0,0
2,Ampleforth,2020-01-01,0.01173444541554875681,0,0.00069998763164798702,0,0,0,0,0,0,0,0
3,Balancer,2020-01-01,0.06413700305234542721,0.12615133491726173341,0.02007276093596930928,0,0,0,0,0,0,0,0
4,Basic Attention Token,2020-01-01,0.01367299925963052625,0.05064107404201471532,0.00103998753524218743,0,0,0,0,0,0,0,0


In [ ]:
processed_df.tail()

,name,Date,variableBorrowRate,stableBorrowRate,liquidityRate,avg1DaysVariableBorrowRate,avg7DaysVariableBorrowRate,avg30DaysVariableBorrowRate,avg91DaysVariableBorrowRate,avg1DaysLiquidityRate,avg7DaysLiquidityRate,avg30DaysLiquidityRate,avg91DaysLiquidityRate
23152,Uniswap,2022-01-17,0.00291646488205642672,0.00500486025095391799,0.00004361638567585159,0.0013309019567712,0.00165510275175792,0.00697860423535104,0.01198294768929888,0.0000124112340504,0.0000973967474016,0.00112060533503952,0.0012715948222128
23153,Wrapped BTC,2022-01-17,0.0027003402777472723,0.03393392954477999304,0.00004786486305112304,0.00259605294201072,0.00257260511774304,0.00284381754494544,0.00422206858526688,0.00004441405609728,0.00006093974728656,0.000085208174856,0.00027381925666512
23154,Wrapped Ether,2022-01-17,0.00377203062534592459,0.03531545521627415242,0.00010900715416202143,0.00354861258748128,0.00335756902267008,0.00299605842222624,0.00296651193354288,0.00011280910299984,0.00010880965134576,0.00011550117111696,0.00102930890517072
23155,renFIL,2022-01-17,0.03543473505110974439,0,0.00904803893874110808,0.11264125889374416,0.0354813909746256,0.02863690059540384,0.0187655795709624,0.02716599882336192,0.00818588816801136,0.0060978658011696,0.00317495063454048
23156,yearn.finance,2022-01-17,0.01335556279827031463,0.01913384062674426715,0.00090564335735365062,0.0139207941059976,0.01236548092560048,0.0099579768707304,0.02389458174805872,0.00099768456305856,0.00079795715698656,0.00054372065476896,0.0044221847551632


In [ ]:
processed_df.to_pickle('/content/gdrive/MyDrive/AaveData/processed_df.pkl')

In [ ]:
processed_df

,name,Date,variableBorrowRate,stableBorrowRate,liquidityRate,avg1DaysVariableBorrowRate,avg7DaysVariableBorrowRate,avg30DaysVariableBorrowRate,avg91DaysVariableBorrowRate,avg1DaysLiquidityRate,avg7DaysLiquidityRate,avg30DaysLiquidityRate,avg91DaysLiquidityRate
0,0x Protocol Token,2020-01-01,0.00246641078358969225,0.03408720076640415241,0.00040890023444781115,0,0,0,0,0,0,0,0
1,Aave Token,2020-01-01,0,0,0,0,0,0,0,0,0,0,0
2,Ampleforth,2020-01-01,0.01173444541554875681,0,0.00069998763164798702,0,0,0,0,0,0,0,0
3,Balancer,2020-01-01,0.06413700305234542721,0.12615133491726173341,0.02007276093596930928,0,0,0,0,0,0,0,0
4,Basic Attention Token,2020-01-01,0.01367299925963052625,0.05064107404201471532,0.00103998753524218743,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23152,Uniswap,2022-01-17,0.00291646488205642672,0.00500486025095391799,0.00004361638567585159,0.0013309019567712,0.00165510275175792,0.00697860423535104,0.01198294768929888,0.0000124112340504,0.0000973967474016,0.00112060533503952,0.0012715948222128
23153,Wrapped BTC,2022-01-17,0.0027003402777472723,0.03393392954477999304,0.00004786486305112304,0.00259605294201072,0.00257260511774304,0.00284381754494544,0.00422206858526688,0.00004441405609728,0.00006093974728656,0.000085208174856,0.00027381925666512
23154,Wrapped Ether,2022-01-17,0.00377203062534592459,0.03531545521627415242,0.00010900715416202143,0.00354861258748128,0.00335756902267008,0.00299605842222624,0.00296651193354288,0.00011280910299984,0.00010880965134576,0.00011550117111696,0.00102930890517072
23155,renFIL,2022-01-17,0.03543473505110974439,0,0.00904803893874110808,0.11264125889374416,0.0354813909746256,0.02863690059540384,0.0187655795709624,0.02716599882336192,0.00818588816801136,0.0060978658011696,0.00317495063454048
